In [2]:
import pymc3 as pm

X = [5, 12, 14, 10, 12]

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [6]:
with pm.Model() as my_first_model:
    
    theta = pm.InverseGamma('theta', alpha=10, beta=100)
    X_obs = pm.Exponential('X_obs', lam=1 / theta, observed=X)

In [8]:
map_estimate = pm.find_MAP(model=my_first_model, start={'theta':10})
print(map_estimate)


{'theta_log__': array(2.25784936), 'theta': array(9.56250158)}


In [11]:
help(pm.Normal)

Help on class Normal in module pymc3.distributions.continuous:

class Normal(pymc3.distributions.distribution.Continuous)
 |  Normal(name, *args, **kwargs)
 |  
 |  Univariate normal log-likelihood.
 |  
 |  The pdf of this distribution is
 |  
 |  .. math::
 |  
 |     f(x \mid \mu, \tau) =
 |         \sqrt{\frac{\tau}{2\pi}}
 |         \exp\left\{ -\frac{\tau}{2} (x-\mu)^2 \right\}
 |  
 |  Normal distribution can be parameterized either in terms of precision
 |  or standard deviation. The link between the two parametrizations is
 |  given by
 |  
 |  .. math::
 |  
 |     \tau = \dfrac{1}{\sigma^2}
 |  
 |  .. plot::
 |  
 |      import matplotlib.pyplot as plt
 |      import numpy as np
 |      import scipy.stats as st
 |      plt.style.use('seaborn-darkgrid')
 |      x = np.linspace(-5, 5, 1000)
 |      mus = [0., 0., 0., -2.]
 |      sigmas = [0.4, 1., 2., 0.4]
 |      for mu, sigma in zip(mus, sigmas):
 |          pdf = st.norm.pdf(x, mu, sigma)
 |          plt.plot(x, pdf, labe

In [14]:
import numpy as np
x=np.array([10, 20, 30, 29, 28, 30, 32, 16, 24, 26, 32, 40, 50, 31, 38, 35, 28])
from scipy import stats, optimize
def logprod(mu):
    return np.sum(-1*stats.poisson.logpmf(x,mu)) #-1把求解极大值变为极小值
result=optimize.minimize_scalar(logprod)
print(result)  # x=29.3

     fun: 68.30525849253605
    nfev: 23
     nit: 9
 success: True
       x: 29.352940747484435


i:\ProgramFiles\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:2341: RuntimeWarning: invalid value encountered in double_scalars
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom


In [ ]:
import pymc3 as pm
with pm.Model() as m1:      # 建立模型
  mu=pm.Uniform('mu',lower=0,upper=100)  # 指定求解参数的分布
  x_obs=pm.Poisson('x',mu=mu,observed=x)  # 这里求解似然值，而不是后验分布
  start=pm.find_MAP()  # 参数初猜
  step=pm.Metropolis() # 指定抽样方法
  trace=pm.sample(10000, start=start, step=step, cores=1) # 抽样过程
  # cores=1为了pymc3在Windows下正常使用
  pm.traceplot(trace)
  pm.autocorrplot(trace)
  pm.summary(trace)  # x=29.4, 95%CI:26.7~31.8
